# Deep Learning
Our deep learning model we uses get_dummies to encode our categorical data (genre and director). We created the test train split (default 75/25) and scaled our data using the standard scaler.  We used to_categorical to encode our target (rating class). Our model included 2 hidden layers with 100 nodes each. The accuracy on this model was 58%.

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from config import username, password
from numpy.random import seed
seed(42)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import tensorflow
tensorflow.keras.__version__
import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestClassifier


c:\users\jagno\.conda\envs\pythondata\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\jagno\.conda\envs\pythondata\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\jagno\.conda\envs\pythondata\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\jagno\.conda\envs\pythondata\lib\site-packages\tensorflow\

## Data Preprocessing

In [2]:
# Read in movie csv
movies = pd.read_csv("./Resources/imdb_final.csv")
movies.head()

,imdb_title_id,title,year,genre,duration,country,director,production_company,budget,total_votes,...,allover45,males,males18to29,males30to44,malesover45,females,females18to29,females30to44,femalesover45,rating_class
0,tt0035423,Kate & Leopold,2001,Comedy,118,USA,James Mangold,Konrad Pictures,48000000,75298,...,6.4,6.3,6.5,6.2,6.3,6.6,6.7,6.4,6.7,Good
1,tt0113026,The Fantasticks,2000,Musical,86,USA,Michael Ritchie,Michael Ritchie Productions,10000000,1082,...,5.4,5.3,5.4,5.4,5.3,5.7,5.6,5.8,5.6,Good
2,tt0118589,Glitter,2001,Drama,104,USA,Vondie Curtis-Hall,Twentieth Century Fox,22000000,20959,...,2.1,1.9,2.1,1.9,2.0,2.9,3.2,2.7,2.5,Bad
3,tt0118652,The Attic Expeditions,2001,Comedy,100,USA,Jeremy Kasten,Tse Tse Fly Productions,1000000,1588,...,4.6,5.0,4.7,5.1,4.6,4.8,5.8,4.6,4.6,Good
4,tt0120467,Vulgar,2000,Crime,87,USA,Bryan Johnson,Chango Productions,120000,3852,...,5.0,5.3,6.1,5.3,5.1,5.2,5.2,5.3,4.6,Good


In [3]:
# # Set vote column to integer type
# movies["median_vote"] = movies["median_vote"].astype(int)
# movies["median_vote"].unique()

In [4]:
# Set values for X and y
X = movies[["year", "genre", "duration", "director", "budget"]]
y = movies["rating_class"].values.reshape(-1,1)
print(X.shape, y.shape)

(5060, 5) (5060, 1)


In [5]:
data = X.copy()
data

,year,genre,duration,director,budget
0,2001,Comedy,118,James Mangold,48000000
1,2000,Musical,86,Michael Ritchie,10000000
2,2001,Drama,104,Vondie Curtis-Hall,22000000
3,2001,Comedy,100,Jeremy Kasten,1000000
4,2000,Crime,87,Bryan Johnson,120000
...,...,...,...,...,...
5055,2019,Comedy,84,Jon Lucas,5000000
5056,2019,Drama,94,Dan Sallitt,95000
5057,2019,Action,84,Glenn Miller,100000
5058,2019,Action,92,Keoni Waxman,3000000


In [6]:
# # Dummy Encoding for genre column
data_binary_encoded = pd.get_dummies(data, columns= ["genre", "director"])
data_binary_encoded.head()

,year,duration,budget,genre_Action,genre_Adventure,genre_Animation,genre_Biography,genre_Comedy,genre_Crime,genre_Drama,...,director_Zack Snyder,director_Zackary Adler,director_Zak Knutson,director_Zak Penn,director_Zebediah De Soto,director_Zia Mojabi,director_Ziad H. Hamzeh,director_Zoe Quist,director_Zoran Lisinac,director_mink
0,2001,118,48000000,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2000,86,10000000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2001,104,22000000,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,2001,100,1000000,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2000,87,120000,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# Create train/test split
X_train, X_test, y_train, y_test = train_test_split(data_binary_encoded, y, random_state=42, stratify= y)


In [8]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train).reshape(-1,1)
encoded_y_test = label_encoder.transform(y_test).reshape(-1,1)
encoded_y_train

array([[1],
       [0],
       [0],
       ...,
       [0],
       [2],
       [2]])

In [9]:
# Create StandardScaler model and fit to training data
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(encoded_y_train)

# Transform training and testing data using X_scaler and y_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(encoded_y_train)
y_test_scaled = y_scaler.transform(encoded_y_test)
print(X_train_scaled[0])

[-1.76326177  0.8312829  -0.47813651 ...  0.         -0.01623496
 -0.01623496]


In [10]:
from tensorflow.keras.utils import to_categorical
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)
y_train_categorical[0]

array([0., 1., 0.], dtype=float32)

## Create Deep Learning Model

In [11]:
# Create deep learning model
# Visible layer with 3537 inputs
# Two hidden layers with 100 nodes
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
deep_model = Sequential()
deep_model.add(Dense(units=100, activation= "relu", input_dim=3537))
deep_model.add(Dense(units=100, activation= "relu"))
deep_model.add(Dense(units=100, activation= "relu"))
deep_model.add(Dense(units=3, activation= "softmax"))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [12]:
# Compile and fit the model
deep_model.compile(optimizer = "adam", loss= "mse", metrics=["accuracy"])
deep_model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               353800    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 303       
Total params: 374,303
Trainable params: 374,303
Non-trainable params: 0
_________________________________________________________________


In [13]:
deep_model.fit(X_train_scaled, y_train_categorical, epochs=100, shuffle=True, verbose=2)

Epoch 1/100
3795/3795 - 1s - loss: 0.1701 - acc: 0.6464
Epoch 2/100
3795/3795 - 1s - loss: 0.0841 - acc: 0.8264
Epoch 3/100
3795/3795 - 1s - loss: 0.0440 - acc: 0.9067
Epoch 4/100
3795/3795 - 1s - loss: 0.0339 - acc: 0.9254
Epoch 5/100
3795/3795 - 1s - loss: 0.0310 - acc: 0.9270
Epoch 6/100
3795/3795 - 1s - loss: 0.0286 - acc: 0.9347
Epoch 7/100
3795/3795 - 1s - loss: 0.0277 - acc: 0.9357
Epoch 8/100
3795/3795 - 1s - loss: 0.0257 - acc: 0.9402
Epoch 9/100
3795/3795 - 1s - loss: 0.0233 - acc: 0.9457
Epoch 10/100
3795/3795 - 1s - loss: 0.0223 - acc: 0.9465
Epoch 11/100
3795/3795 - 1s - loss: 0.0211 - acc: 0.9515
Epoch 12/100
3795/3795 - 1s - loss: 0.0202 - acc: 0.9531
Epoch 13/100
3795/3795 - 1s - loss: 0.0189 - acc: 0.9557
Epoch 14/100
3795/3795 - 1s - loss: 0.0180 - acc: 0.9557
Epoch 15/100
3795/3795 - 1s - loss: 0.0169 - acc: 0.9570
Epoch 16/100
3795/3795 - 1s - loss: 0.0160 - acc: 0.9605
Epoch 17/100
3795/3795 - 1s - loss: 0.0157 - acc: 0.9626
Epoch 18/100
3795/3795 - 1s - loss: 0.01

## Quantify the Model

In [14]:
model_loss, model_accuracy = deep_model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}")
print(f"Deep Neural Network - Accuracy: {model_accuracy}")

1265/1265 - 0s - loss: 0.2033 - acc: 0.6008
Deep Neural Network - Loss: 0.20325156895539506
Deep Neural Network - Accuracy: 0.6007905006408691


## Make Predictions

In [15]:
encoded_predictions = deep_model.predict_classes(X_test_scaled)
prediction_labels = label_encoder.inverse_transform(encoded_predictions)
print(encoded_predictions[:10])
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {y_test[:10]}")

[0 2 2 2 2 0 2 2 2 2]
Predicted classes: ['Bad' 'Good' 'Good' ... 'Good' 'Excellent' 'Good']
Actual Labels: [['Good']
 ['Good']
 ['Excellent']
 ['Excellent']
 ['Good']
 ['Good']
 ['Good']
 ['Good']
 ['Good']
 ['Good']]


In [16]:
# CLassification report
from sklearn.metrics import classification_report
print(classification_report(y_test, prediction_labels))

              precision    recall  f1-score   support

         Bad       0.42      0.54      0.47       270
   Excellent       0.28      0.31      0.29       167
        Good       0.77      0.68      0.72       828

    accuracy                           0.60      1265
   macro avg       0.49      0.51      0.50      1265
weighted avg       0.63      0.60      0.61      1265



## Save Model

In [17]:
deep_model.save("dl.h5")